In [48]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
print('Libraries imported')

Libraries imported


###### In this Jupyter Notebook, we will illsutrate venues(Gym/Fitness) in 4 different cities across China: Beijing, Shanghai, Chengdu, Chongqing

###### The search engine to investigate no.of venues we use is Foursquare, and the API details of the host is shown below

In [49]:
#Foursquare API (free trial account)
CLIENT_ID = 'P1X3FJMKPO4DWCK1FDOBLB51PF4QPP4C1OQIQENXUREVCXJX'
CLIENT_SECRET = 'VD1ZTKQQOOKTIONJ5TGCOVXFDK25YYOBOINQ13J0NET0YGX0'
VERSION = '20200727'

###### First, lets get the [Lat, Lon] coordinates of these cities so we can generate a map via Folium

In [50]:
geolocator_Beijing = Nominatim(user_agent = 'TayTay')
city_1 = 'Beijing'
country = 'China'
location_Beijing = geolocator_Beijing.geocode(city_1 + ',' + country)
print(location_Beijing.address)
print(location_Beijing.latitude, location_Beijing.longitude)

北京市, 东城区, 北京市, 100010, China 中国
39.906217 116.3912757


In [51]:
geolocator_Shanghai = Nominatim(user_agent = 'TayTay')
city_2 = 'Shanghai'
country = 'China'
location_Shanghai = geolocator_Shanghai.geocode(city_2 + ',' + country)
print(location_Shanghai.address)
print(location_Shanghai.latitude, location_Shanghai.longitude)

上海市, 黄浦区, 上海市, 200010, China 中国
31.2252985 121.4890497


In [52]:
geolocator_Chengdu = Nominatim(user_agent = 'TayTay')
city_3 = 'Chengdu'
country = 'China'
location_Chengdu = geolocator_Chengdu.geocode(city_3 + ',' + country)
print(location_Chengdu.address)
print(location_Chengdu.latitude, location_Chengdu.longitude)

成都市, 四川省, China 中国
30.6624205 104.0633219


In [53]:
geolocator_Chongqing = Nominatim(user_agent = 'TayTay')
city_4 = 'Chongqing'
country = 'China'
location_Chongqing = geolocator_Chongqing.geocode(city_4 + ',' + country)
print(location_Chongqing.address)
print(location_Chongqing.latitude, location_Chongqing.longitude)

重庆市, 渝中区 (Yuzhong), 重庆市中心, 重庆市, 400014, China 中国
29.5585712 106.5492822


###### For map display, we will use Folium, and we round the latitude and longitude of cities to 2 decimal places for convience

In [54]:
#round coordinates into 2 decimal places
latitude_0 = location_Beijing.latitude
longitude_0 = location_Beijing.longitude
longitude_0_0 = round(longitude_0,2)
latitude_0_0 = round(latitude_0,2)

latitude_1 = location_Shanghai.latitude
longitude_1 = location_Shanghai.longitude
longitude_1_1 = round(longitude_1,2)
latitude_1_1 = round(latitude_1,2)

latitude_2 = location_Chengdu.latitude
longitude_2 = location_Chengdu.longitude
longitude_2_2 = round(longitude_2,2)
latitude_2_2 = round(latitude_2,2)

latitude_3 = location_Chongqing.latitude
longitude_3 = location_Chongqing.longitude
longitude_3_3 = round(longitude_3,2)
latitude_3_3 = round(latitude_3,2)

###### For map display Folium is used, as Folium does not have a language parameter, if have doubt regards to city coordinate, please feel free to check the [lat, lon] coordinates on a map of your choice. Google, Baidu, Gaode ect.

In [55]:
#display maps
map_Beijing = folium.Map(location=[latitude_0_0, longitude_0_0], zoom_start=11)
map_Beijing

In [56]:
map_Shanghai = folium.Map(location=[latitude_1_1, longitude_1_1], zoom_start=11)
map_Shanghai

In [57]:
map_Chengdu = folium.Map(location=[latitude_2_2, longitude_2_2], zoom_start=11)
map_Chengdu

In [58]:
map_Chongqing = folium.Map(location=[latitude_2_2, longitude_2_2], zoom_start=11)
map_Chongqing

##### Now we use the foursquare API to search for venues(gym/fitness) in each city 

In [59]:
import requests

LIMIT = 1000
cities = ['Beijing,Beijing', 'Shanghai, Shanghai', 'Chengdu, Sichuan', 'Chongqing, Chongqing']
results_gym = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d175941735") # Gym/Fitness PLACE CATEGORY ID
    results_gym[city] = requests.get(url).json()

In [60]:
results_gym[cities[0]]

{'meta': {'code': 200, 'requestId': '5f1ebbfbcbd37754cc7ab5b3'},
 'response': {'geocode': {'cc': 'CN',
   'center': {'lat': 39.9075, 'lng': 116.39723},
   'displayString': 'Beijing, Beijing Shi, China',
   'geometry': {'bounds': {'ne': {'lat': 41.05875499151405,
      'lng': 117.49936797940963},
     'sw': {'lat': 39.44236100189344, 'lng': 115.41642409091048}}},
   'longId': '72057594039744606',
   'slug': 'beijing',
   'what': '',
   'where': 'beijing beijing'},
  'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4da2c281540ea1cd759382de-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d175941735',
         'name': 'Gym / Fitness Center',
         'pluralName': 'Gyms or Fitness Centers',
         'primary': True,

##### Seeing from the JSON results of each search, we will extract info relates to the [name, address, location[latitude, longitude], summary], then we put the results into a pandas dataframe.

In [61]:
df_venues={}
for city in cities:
    venues = json_normalize(results_gym[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [62]:
maps_gyms={}
for city in cities:
    city_lat = np.mean([results_gym[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_gym[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_gym[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_gym[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps_gyms[city] = folium.Map(location=[city_lat, city_lng], zoom_start=6)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            parse_html=False).add_to(maps_gyms[city])  
    print(f"Total number of Gym/Fitness places in {city} = ", results_gym[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Gym/Fitness places in Beijing,Beijing =  56
Showing Top 100
Total number of Gym/Fitness places in Shanghai, Shanghai =  104
Showing Top 100
Total number of Gym/Fitness places in Chengdu, Sichuan =  8
Showing Top 100
Total number of Gym/Fitness places in Chongqing, Chongqing =  9
Showing Top 100


###### Display no.of gym/fitness venues in cities in order of Beijing as an example

In [63]:
maps_gyms[cities[0]]

###### Lets try to get the JSON results into a panda dataframe, here we use Beijing and Shanghai as examples

In [64]:
#function to get category type
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
gym_beijing = results_gym[cities[0]]['response']['groups'][0]['items']
nearby_gym_beijing = json_normalize(gym_beijing)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_gym_beijing = nearby_gym_beijing.loc[:, filtered_columns]
nearby_gym_beijing['venue.categories'] = nearby_gym_beijing.apply(get_category_type, axis=1)
nearby_gym_beijing.columns = [col.split('.')[-1] for col in nearby_gym_beijing.columns]

In [67]:
gym_shanghai = results_gym[cities[1]]['response']['groups'][0]['items']
nearby_gym_shanghai = json_normalize(gym_shanghai)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_gym_shanghai = nearby_gym_shanghai.loc[:, filtered_columns]
nearby_gym_shanghai['venue.categories'] = nearby_gym_shanghai.apply(get_category_type, axis=1)
nearby_gym_shanghai.columns = [col.split('.')[-1] for col in nearby_gym_shanghai.columns]

##### Now we want to look at how many sport shops are there in each city

In [68]:
import requests
LIMIT = 10000
results_shops = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1f2941735") # Sports shopes CATEGORY ID
    results_shops[city] = requests.get(url).json()

In [69]:
results_shops[cities[0]]

{'meta': {'code': 200, 'requestId': '5f1ebde35384763b4f47923b'},
 'response': {'geocode': {'cc': 'CN',
   'center': {'lat': 39.9075, 'lng': 116.39723},
   'displayString': 'Beijing, Beijing Shi, China',
   'geometry': {'bounds': {'ne': {'lat': 41.05875499151405,
      'lng': 117.49936797940963},
     'sw': {'lat': 39.44236100189344, 'lng': 115.41642409091048}}},
   'longId': '72057594039744606',
   'slug': 'beijing',
   'what': '',
   'where': 'beijing beijing'},
  'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b058794f964a5204d9b22e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f2941735',
         'name': 'Sporting Goods Shop',
         'pluralName': 'Sporting Goods Shops',
         'primary': 

In [70]:
shops_beijing = results_shops[cities[0]]['response']['groups'][0]['items']
nearby_shops_beijing = json_normalize(shops_beijing)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_shops_beijing = nearby_shops_beijing.loc[:, filtered_columns]
nearby_shops_beijing['venue.categories'] = nearby_shops_beijing.apply(get_category_type, axis=1)
nearby_shops_beijing.columns = [col.split('.')[-1] for col in nearby_shops_beijing.columns]

In [27]:
results_shops[cities[1]]

{'meta': {'code': 200, 'requestId': '5f1e9a801db3e514e98c23da'},
 'response': {'geocode': {'cc': 'CN',
   'center': {'lat': 31.22222, 'lng': 121.45806},
   'displayString': 'Shanghai, Shanghai Shi, China',
   'geometry': {'bounds': {'ne': {'lat': 31.87021999293653,
      'lng': 121.98831988297854},
     'sw': {'lat': 30.686803039798992, 'lng': 120.87753211035766}}},
   'longId': '72057594039724172',
   'slug': 'shanghai',
   'what': '',
   'where': 'shanghai shanghai'},
  'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-51f4fa20498e4b932e462627-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f2941735',
         'name': 'Sporting Goods Shop',
         'pluralName': 'Sporting Goods Shops',
         'pri

In [71]:
shops_shanghai = results_shops[cities[1]]['response']['groups'][0]['items']
nearby_shops_shanghai = json_normalize(shops_shanghai)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_shops_shanghai = nearby_shops_shanghai.loc[:, filtered_columns]
nearby_shops_shanghai['venue.categories'] = nearby_shops_shanghai.apply(get_category_type, axis=1)
nearby_shops_shanghai.columns = [col.split('.')[-1] for col in nearby_shops_shanghai.columns]

In [72]:
df_shops={}
for city in cities:
    shops = json_normalize(results_shops[city]['response']['groups'][0]['items'])
    df_shops[city] = shops[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_shops[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [114]:
maps_shops={}

for city in cities:
    city_lat = np.mean([results_shops[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_shops[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_shops[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_shops[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    
    maps_shops[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(df_shops[city]['Lat'], df_shops[city]['Lng'], df_shops[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(maps_shops[city]) 
        
    print(f"Total number of sports shop places in {city} = ", results_shops[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of sports shop places in Beijing,Beijing =  21
Showing Top 100
Total number of sports shop places in Shanghai, Shanghai =  43
Showing Top 100
Total number of sports shop places in Chengdu, Sichuan =  16
Showing Top 100
Total number of sports shop places in Chongqing, Chongqing =  9
Showing Top 100


In [111]:
maps_shops[cities[1]]

In [86]:
maps_gyms[cities[1]]

In [83]:
no_of_gyms_Beijing =  results_gym[cities[0]]['response']['totalResults']
no_of_shops_Beijing = results_shops[cities[0]]['response']['totalResults']
gym_shops_ratio_Beijing = no_of_gyms_Beijing/no_of_shops_Beijing
print(no_of_gyms_Beijing, no_of_shops_Beijing, gym_shops_ratio_Beijing)

56 21 2.6666666666666665


In [84]:
no_of_gyms_Shanghai =  results_gym[cities[1]]['response']['totalResults']
no_of_shops_Shanghai = results_shops[cities[1]]['response']['totalResults']
gym_shops_ratio_Shanghai = no_of_gyms_Shanghai/no_of_shops_Shanghai
print(no_of_gyms_Shanghai, no_of_shops_Shanghai, gym_shops_ratio_Shanghai)

104 43 2.4186046511627906


In [96]:
df_venues[cities[1]]

,Name,Address,Lat,Lng
0,Z&B Fitness,14/F 1219 Changle Rd,31.216626,121.437774
1,The Clinic,"501, Building B, 98-118 Jiashan Rd | 徐汇区嘉善路98-...",31.213464,121.454507
2,Pure Fitness,999 Huaihai M Rd | 淮海中路999号,31.217676,121.453586
3,Z&B Fitness Studio,278 Shaanxi N Rd | 陕西北路278号,31.230207,121.449826
4,Kerry Sports,1388 Huamu Rd,31.215612,121.559629
...,...,...,...,...
95,Pure Yoga,3/F-35 1266 Nanjing W Rd | 南京西路1266号恒隆广场3楼,31.229978,121.448635
96,9 Clouds Space 九霄会馆,4f 17 Xiangyang Road,31.221949,121.450069
97,Drishti Vinyasa Yoga Shala,NaN,31.225097,121.424711
98,三和花园游泳馆,1155号 Kangding Rd,31.231690,121.434755


In [118]:
for city in cities:
    city_lat = np.mean([results_shops[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_shops[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_shops[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_shops[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    

    map_SH = maps_gyms[cities[1]]

    for lat, lng, label in zip(df_shops[city]['Lat'], df_shops[city]['Lng'], df_shops[city]['Name']):
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=0.7,
                parse_html=False).add_to(map_SH) 

    for lat, lng, label in zip(df_shops[city]['Lat'], df_shops[city]['Lng'], df_shops[city]['Name']):
            label = folium.Popup(label, parse_html=True)
            folium.Circle(
                [31.229978, 121.448635],
                radius=1000,
                popup=label,
                color='#FF7F0F',
                fill=False).add_to(map_SH)
map_SH